# Create useable csv files from NSRR datasets

In [2]:
# 3rd party packages:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog,messagebox
import ast
# local packages:
import os

# personal packages:


In [2]:
# get folder path
root=tk.Tk()
root.attributes("-topmost", True) 
root.withdraw() # we don't want a full GUI, so keep the root window from appearing

folder_path = filedialog.askdirectory(parent=root,title='Please select a directory with the o1 folder')

if "o1" not in folder_path:
    if os.path.exists(folder_path + "/o1"):
        folder_path = folder_path + "/o1"
        print(folder_path)
    else:
        print("No o1 folder found")
        raise FileNotFoundError
else:
    print(folder_path)


C:/Users/tomo9/Documents/00_School/00_Thesis/HypnoGAN/Data/o1


In [5]:
# get all files in folder
files = os.listdir(folder_path)
print(files)
used_cols = [4]

new_names_df = pd.DataFrame(columns=['File name','Sleeping_stage', 'length', 'additional_info'])


for index,file in enumerate(files):
    #print(index,file)
    
    temp_df = pd.read_csv(os.path.join(folder_path,file,'STAGE_E.txt'),skiprows=1 ,usecols=used_cols,delimiter='\t',names=['stage'])

    # transform stage to values
    mapping= {'w': int(1), 'r': int(2), 'n1': int(3), 'n2': int(4), 'n3': int(5)}
    temp_df['value'] = temp_df['stage'].apply(lambda x: x.lower()).map(mapping)
    print(type(temp_df['value'][0]))
    #temp_df['value'] = temp_df['value'].apply(lambda x: x.astype(int))
    #drop row if value is nan
    #'time': temp_df['time'].values.tolist(),
    row = {
        'File name': file,
        
        'Sleeping_stage': temp_df['value'].values.tolist(),
    }
    if temp_df['value'].isna().any().any() == False:
        new_names_df.loc[index] = row
#add the length of the sleep stage array to the dataframe
new_names_df['length'] = new_names_df['Sleeping_stage'].apply(lambda x: len(x))
#new_names_df['time'] = new_names_df['time'].apply(lambda x: float(x))
#new_names_df['Sleeping stage'] = new_names_df['Sleeping stage'].apply(lambda x: float(x))
#add additional info
# Unused for now
#TODO: add additional info

#new_names_df["Sleeping stage"] = new_names_df["Sleeping stage"].apply(lambda x: x.astype(int))
#new_names_df["time"] = new_names_df["time"].apply(lambda x: x.astype(float))
#check if all files are in the dataframe
if len(files) == len(new_names_df):
    print("All files are in the dataframe")
else:
    print("Not all files are in the dataframe")
    print("files: ",len(files))
    print("df: ",len(new_names_df))

['chat-baseline-nonrandomized-300003', 'chat-baseline-nonrandomized-300005', 'chat-baseline-nonrandomized-300006', 'chat-baseline-nonrandomized-300009', 'chat-baseline-nonrandomized-300010', 'chat-baseline-nonrandomized-300011', 'chat-baseline-nonrandomized-300012', 'chat-baseline-nonrandomized-300016', 'chat-baseline-nonrandomized-300017', 'chat-baseline-nonrandomized-300018', 'chat-baseline-nonrandomized-300020', 'chat-baseline-nonrandomized-300022', 'chat-baseline-nonrandomized-300023', 'chat-baseline-nonrandomized-300027', 'chat-baseline-nonrandomized-300028', 'chat-baseline-nonrandomized-300029', 'chat-baseline-nonrandomized-300030', 'chat-baseline-nonrandomized-300032', 'chat-baseline-nonrandomized-300034', 'chat-baseline-nonrandomized-300035', 'chat-baseline-nonrandomized-300040', 'chat-baseline-nonrandomized-300044', 'chat-baseline-nonrandomized-300045', 'chat-baseline-nonrandomized-300046', 'chat-baseline-nonrandomized-300047', 'chat-baseline-nonrandomized-300048', 'chat-basel

In [6]:
# Change name here if you want to save to a different file
save_filename = 'generated_dataset_CHAT_non.csv'
popup = messagebox.askyesno(parent=root,title=f"Append to {save_filename}",message=f"Do you want to append to existing {save_filename} file?")
print(popup)
if popup:
    print(f"Appending to {save_filename}")
    if os.path.exists(save_filename):
        print(f"{save_filename} exists")
        new_names_df.to_csv(save_filename,index=False,mode='a',header=False,decimal='.',sep=';')
    else:
        print(f"{save_filename} does not exist")
        print(f"Creating {save_filename}")
        new_names_df.to_csv(save_filename,index=False,decimal='.',sep=';')
else:
    print("Creating dataset.csv")
    new_names_df.to_csv(save_filename,index=False,sep=';',decimal='.')

False
Creating dataset.csv


# Merge csv files

In [1]:
def string_to_int_array(s):
    return [int(x) for x in ast.literal_eval(s)]

In [4]:
file_1 = 'generated_dataset_ABC.csv'
file_2 = 'generated_dataset_SHHS.csv'
file_3 = 'generated_dataset_CHAT.csv'
output_file = 'generated_dataset.csv'

df_1 = pd.read_csv(file_1,sep=';')
df_2 = pd.read_csv(file_2,sep=';')
df_3 = pd.read_csv(file_3,sep=';')

merged_df = pd.concat([df_1, df_2,df_3])
merged_df.Sleeping_stage = merged_df.Sleeping_stage.apply(string_to_int_array)

merged_df.to_csv(output_file, index=False,decimal='.',sep=';')

ValueError: malformed node or string on line 1: <ast.Name object at 0x0000011EA8F43250>

In [ ]:
import pandas as pd
filename = "generated_dataset_SHHS.csv"
dataset = pd.read_csv(filename,sep=';',index_col=0)
# transform Sleeping stage to int
dataset['Sleeping_stage'] = dataset['Sleeping_stage'].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
dataset.to_csv(filename,sep=';')
